In [5]:
import pandas as pd
import numpy as np
import cv2
from tqdm import tqdm
from os import listdir
from os.path import isfile, join


## function

func qui retourne le nb de fichier dans un path donnée

In [1]:
import pathlib
def countNb(path) :
    initial_count = 0
    for path in pathlib.Path(path).iterdir():
        if path.is_file():
            initial_count += 1

    return(initial_count)

indice des points de jointure (prendre en considération -1 pour chaque point dans le Df)  
{1,  "Nose"},  
{2,  "Neck"},  
{3,  "RShoulder"},  
{4,  "RElbow"},  
{5,  "RWrist"},  
{6,  "LShoulder"},  
{7,  "LElbow"},  
{8,  "LWrist"},  
{9,  "RHip"},  
{10, "RKnee"},  
{11, "RAnkle"},  
{12, "LHip"},  
{13, "LKnee"},  
{14, "LAnkle"},  
{15, "REye"},  
{16, "LEye"},  
{17, "REar"},  
{18, "LEar"},  

In [2]:
# import numpy as np
# def abdomen(xs,ys) :
#     xs_abdomen=pd.concat([xs[1],xs[5],xs[8],xs[11]])

#     xs_abdomen=sum(xs_abdomen) / 4
#     ys_abdomen=pd.concat([ys[2],ys[5],ys[8],ys[11]])

#     ys_abdomen=sum(ys_abdomen) /4

#     return xs_abdomen,ys_abdomen

funct qui prend comme parametre 2 df (xs->les cordonné X des point de jointure pour chaque frame , ys les cordonné de Y)
retourne 2 pt qui sont relative xs_head,ys_head de chaque frame en utlisant des point de jointure   
0->Nose  
14->REye  
15->LEye  
16->REar  
17->LEar  

L représente  Left   
R représente  Right

In [54]:
def head(xs,ys) :
    xs_head=pd.concat([xs[0],xs[14],xs[15],xs[16],xs[17]])
    
   
        
    xs_head=sum(xs_head) / np.count_nonzero(xs_head)
    
    ys_head=pd.concat([ys[0],ys[14],ys[15],ys[16],ys[17]])
    
   

    ys_head=sum(ys_head) / np.count_nonzero(ys_head)

    return xs_head,ys_head

nous avons utilisé L'échelle de chaque pose  pour extraire des patchs de taille cohérente. qui  a été obtenue en divisant la hauteur de la pose (en pixels) par 240 .   

Dans le système de coordonnées de l'image, la hauteur a été calculée en utilisant la différence maximum des coordonnées y entre la tête et les chevilles.  
Une région d'intérêt autour d'une partie du corps a été extraite en ajoutant une marge aux positions des points clés haut-gauche et bas-droit de l'image.   
La marge a été calculée comme suit : C × S, où C a été choisi comme étant de 75 pixels et S comme étant l'échelle .
puis la marge sera * par 0.4

In [4]:

def pose(df) :
#     l pose between the head and ankles.
# ankle indice 14 et 11
#  head indice mte3ou .
    
    xs = df[df.index==0]
    ys = df[df.index==1]
    
    a,b=head(xs,ys)
    # a, b sont relatives à la position du couple de position (x, y) de la tete
    pose = b-ys[13]
    # la pose est la distance entre la cheville noté ys[13] et la tete
    scale =pose/240
    c=100
    margin=c*scale
    return pose,scale,margin



on a ou préalable extrait les frame de chaque vidéo est-on les a placé dans un variable intitulé pathImage .
pour grouper les images ont première partie par acteur puis par action nous allons utiliser le path des fichiers Csv qui contiennent les annotations des jointure actions et le placer en suivant la même architecture (Acteur -> action ->image)

## upper body

In [75]:

path =r'C:\Users\amin\extracted patch\Nouveau dossier'
dossier = [f for f in listdir(path) ]
#la variable dossier est une liste qui contient les noms des dossiers des acteurs 
pathImg = r'C:\Users\amin\extracted patch\action'+'\\'
# pathImg c est le path des frames 
for i in tqdm(range (0,10)) :
    pathData = r'C:\Users\amin\extracted patch\Nouveau dossier'+'\\'
    pathData=pathData+ dossier[i]+'\\'
    Data = [f for f in listdir(pathData) ]
    mp=dossier[i]
    for j in range (0,len(Data)) :
        a=pathData
        do=Data[j]
        a=a+Data[j]+'\\'
        do1=do
        if do1=='S10_punching_toLeft_sideView_HD':
            do1='S10_punching_toLeft__ideView_HD'
#         print(a)
        
    ############### C:\Users\amin\extracted patch\data\boxing\S1
        nb=int((countNb(a)/2)+1)
        for n in  range(1,nb):
            df = pd.read_csv (a+do+'-bbox-'+str(n)+'.csv',header=None)


           
            h = int(df[3]/2)
            w = int(df[2])
            # w C'est le point bas-droit du Bbox extraite du fichier csv 
            # h est divisé par 2 car on s'intéresse à la partie supérieure du corps
            # h C'est le point haut-gauche du Bbox extraite extraite du fichier csv

           
           
            image = cv2.imread(pathImg+do1+'_'+str(n)+'.jpg')
            y=int(df[1])
            x=int(df[0])
            # x C'est le point bas-gauche du Bbox extraite du fichier csv
            # y C'est le point haut-droit du Bbox extraite extraite du fichier csv
            
            crop = image[y:y+h, x:x+w]

            cv2.imwrite(r'C:\Users\amin\extracted patch\image\ub//'+mp+'//'+do+'_'+str(n)+'.jpg', crop)
         
            cv2.waitKey(0)


100%|██████████| 1/1 [10:29<00:00, 629.52s/it]


['S3_clapping_HD',
 'S3_hittingBottle_toLeft_HD',
 'S3_hittingBottle_toRight_HD',
 'S3_hittingStick_toLeft_HD',
 'S3_hittingStick_toRight_HD',
 'S3_jogging_backView_HD',
 'S3_jogging_frontView_HD',
 'S3_jogging_toLeft_sideView_HD',
 'S3_jogging_toRight_sideView_HD',
 'S3_kicking_toLeft_HD',
 'S3_kicking_toRight_HD',
 'S3_punching_toLeft_sideView_HD',
 'S3_punching_toRight_sideView_HD',
 'S3_running_backView_HD',
 'S3_running_frontView_HD',
 'S3_running_toLeft_sideView_HD',
 'S3_running_toRight_sideView_HD',
 'S3_stabbing_toLeft_HD',
 'S3_stabbing_toRight_HD',
 'S3_walking_backView_HD',
 'S3_walking_frontView_HD',
 'S3_walking_toLeft_sideView_HD',
 'S3_walking_toRight_sideView_HD',
 'S3_wavingHands_HD']

## full body


In [6]:

path =r'C:\Users\amin\extracted patch\Nouveau dossier'
dossier = [f for f in listdir(path) ]
m=0 
pathImg = r'C:\Users\amin\extracted patch\action'+'\\'
for i in tqdm(range (0,10)) :
    pathData = r'C:\Users\amin\extracted patch\Nouveau dossier'+'\\'
    pathData=pathData+ dossier[i]+'\\'
    Data = [f for f in listdir(pathData) ]
    mp=dossier[i]
    for j in range (0,len(Data)) :
        a=pathData
        do=Data[j]
        a=a+Data[j]+'\\'
        do1=do
        if do1=='S10_punching_toLeft_sideView_HD':
            do1='S10_punching_toLeft__ideView_HD'
#         print(a)
        
    ############### C:\Users\amin\extracted patch\data\boxing\S1
        nb=int((countNb(a)/2)+1)
        for n in  range(1,nb):

            
            df = pd.read_csv (a+do+'-bbox-'+str(n)+'.csv',header=None)

 
            h = int(df[3])
            w = int(df[2])

            # w C'est le point bas-droit du Bbox extraite du fichier csv 
            # h est divisé par 2 car on s'intéresse à la partie supérieure du corps
            # h C'est le point haut-gauche du Bbox extraite extraite du fichier csv

            import numpy as np
            import cv2

      
            image = cv2.imread(pathImg+do1+'_'+str(n)+'.jpg')
            y=int(df[1])
            x=int(df[0])


            # x C'est le point bas-gauche du Bbox extraite du fichier csv
            # y C'est le point haut-droit du Bbox extraite extraite du fichier csv

            
            
            crop = image[y:y+h, x:x+w]

            cv2.imwrite(r'C:\Users\amin\extracted patch\image\fb//'+mp+'//'+do+'_'+str(n)+'.jpg', crop)
         
            cv2.waitKey(0)


 83%|████████▎ | 10/12 [1:55:47<23:09, 694.78s/it] 


IndexError: list index out of range

## Right hand

In [71]:

path =r'C:\Users\amin\extracted patch\Nouveau dossier'
dossier = [f for f in listdir(path) ]
m=0 
pathImg = r'C:\Users\amin\extracted patch\action'+'\\'
for i in tqdm(range (0,10)) :
    pathData = r'C:\Users\amin\extracted patch\Nouveau dossier'+'\\'
    pathData=pathData+ dossier[i]+'\\'
    Data = [f for f in listdir(pathData) ]
    mp=dossier[i]
    for j in range (0,len(Data)) :
        a=pathData
        do=Data[j]
        a=a+Data[j]+'\\'
        do1=do
        if do1=='S10_punching_toLeft_sideView_HD':
            do1='S10_punching_toLeft__ideView_HD'
#         print(a)
        
    ############### extraction du patche
        nb=int((countNb(a)/2)+1)
        for n in  range(1,nb):
            #lecture du fichier 
                df = pd.read_csv (a+do+'-pos_img-'+str(n)+'.csv',header=None)
                
                #ys c'est les points relatifs à laxe x
                #ys c'est les points relatifs à laxe y
                xs = df[df.index==0]
                ys = df[df.index==1]
                #concaténation des pt relatifs à l'extraction due patche Right hand
                a1=pd.concat([xs[3],xs[4],xs[2]])
                b2=pd.concat([ys[3],ys[4],ys[2]])
                
                
                
                
########  teste si un point est egale a 0 car il n est pas visible


                if (xs[2]==0)[0]:

                    a1=pd.concat([xs[3],xs[4]])
                    b2=pd.concat([ys[3],ys[4]])

                if  (xs[3]==0)[0]:

                    a1=pd.concat([xs[2],xs[4]])
                    b2=pd.concat([ys[2],ys[4]])

                if (xs[4]==0)[0]:

                    a1=pd.concat([xs[2],xs[3]])
                    b2=pd.concat([ys[2],ys[3]])
                    
#         __________________________________________________________
                    
                if ((xs[4]==0)[0] )and ( (xs[3]==0)[0]):

                    a1=pd.concat([xs[2]])
                    b2=pd.concat([ys[2]])

                if ((xs[4]==0)[0] )and ((xs[2]==0)[0]):

                    a1=pd.concat([xs[3]])
                    b2=pd.concat([ys[3]])
                    

                if ((xs[2]==0)[0]) and ((xs[3]==0)[0]):

                    a1=pd.concat([xs[4]])
                    b2=pd.concat([ys[4]])
                    
                if ((xs[2]==0)[0]) and ((xs[3]==0)[0]) and  ((xs[4]==0)[0]):
                    df = pd.read_csv (a+do+'-pos_img-'+str(n-1)+'.csv',header=None)

                    xs = df[df.index==0]
                    ys = df[df.index==1]
                    a1=pd.concat([xs[3],xs[4],xs[2]])
                    b2=pd.concat([ys[3],ys[4],ys[2]])




    #                 xs = df[df.index==0]
    #                 ys = df[df.index==1]
    #                 a1=pd.concat([xs[3],xs[2],xs[4]])
    #                 b2=pd.concat([ys[3],ys[2],ys[4]])


                    if (xs[2]==0)[0]:

                        a1=pd.concat([xs[3],xs[4]])
                        b2=pd.concat([ys[3],ys[4]])

                    if  (xs[3]==0)[0]:

                        a1=pd.concat([xs[2],xs[4]])
                        b2=pd.concat([ys[2],ys[4]])

                    if (xs[4]==0)[0]:

                        a1=pd.concat([xs[2],xs[3]])
                        b2=pd.concat([ys[2],ys[3]])

    #         __________________________________________________________

                    if ((xs[4]==0)[0] )and ( (xs[3]==0)[0]):

                        a1=pd.concat([xs[2]])
                        b2=pd.concat([ys[2]])

                    if ((xs[4]==0)[0] )and ((xs[2]==0)[0]):

                        a1=pd.concat([xs[3]])
                        b2=pd.concat([ys[3]])


                    if ((xs[2]==0)[0]) and ((xs[3]==0)[0]):

                        a1=pd.concat([xs[4]])
                        b2=pd.concat([ys[4]])

#                ------------------------------------------------------------------------------------    
                if ((xs[2]==0)[0]) and ((xs[3]==0)[0]) and  ((xs[4]==0)[0]):
                    df = pd.read_csv (a+do+'-pos_img-'+str(n-2)+'.csv',header=None)

                    xs = df[df.index==0]
                    ys = df[df.index==1]
                    a1=pd.concat([xs[3],xs[4],xs[2]])
                    b2=pd.concat([ys[3],ys[4],ys[2]])




    #                 xs = df[df.index==0]
    #                 ys = df[df.index==1]
    #                 a1=pd.concat([xs[3],xs[2],xs[4]])
    #                 b2=pd.concat([ys[3],ys[2],ys[4]])


                    if (xs[2]==0)[0]:

                        a1=pd.concat([xs[3],xs[4]])
                        b2=pd.concat([ys[3],ys[4]])

                    if  (xs[3]==0)[0]:

                        a1=pd.concat([xs[2],xs[4]])
                        b2=pd.concat([ys[2],ys[4]])

                    if (xs[4]==0)[0]:

                        a1=pd.concat([xs[2],xs[3]])
                        b2=pd.concat([ys[2],ys[3]])

    #         __________________________________________________________

                    if ((xs[4]==0)[0] )and ( (xs[3]==0)[0]):

                        a1=pd.concat([xs[2]])
                        b2=pd.concat([ys[2]])

                    if ((xs[4]==0)[0] )and ((xs[2]==0)[0]):

                        a1=pd.concat([xs[3]])
                        b2=pd.concat([ys[3]])


                    if ((xs[2]==0)[0]) and ((xs[3]==0)[0]):

                        a1=pd.concat([xs[4]])
                        b2=pd.concat([ys[4]])

#                ------------------------------------------------------------------------------------    
                if ((xs[2]==0)[0]) and ((xs[3]==0)[0]) and  ((xs[4]==0)[0]):
                    df = pd.read_csv (a+do+'-pos_img-'+str(n-3)+'.csv',header=None)

                    xs = df[df.index==0]
                    ys = df[df.index==1]
                    a1=pd.concat([xs[3],xs[4],xs[2]])
                    b2=pd.concat([ys[3],ys[4],ys[2]])




    #                 xs = df[df.index==0]
    #                 ys = df[df.index==1]
    #                 a1=pd.concat([xs[3],xs[2],xs[4]])
    #                 b2=pd.concat([ys[3],ys[2],ys[4]])


                    if (xs[2]==0)[0]:

                        a1=pd.concat([xs[3],xs[4]])
                        b2=pd.concat([ys[3],ys[4]])

                    if  (xs[3]==0)[0]:

                        a1=pd.concat([xs[2],xs[4]])
                        b2=pd.concat([ys[2],ys[4]])

                    if (xs[4]==0)[0]:

                        a1=pd.concat([xs[2],xs[3]])
                        b2=pd.concat([ys[2],ys[3]])

    #         __________________________________________________________

                    if ((xs[4]==0)[0] )and ( (xs[3]==0)[0]):

                        a1=pd.concat([xs[2]])
                        b2=pd.concat([ys[2]])

                    if ((xs[4]==0)[0] )and ((xs[2]==0)[0]):

                        a1=pd.concat([xs[3]])
                        b2=pd.concat([ys[3]])


                    if ((xs[2]==0)[0]) and ((xs[3]==0)[0]):

                        a1=pd.concat([xs[4]])
                        b2=pd.concat([ys[4]])

#                ------------------------------------------------------------------------------------        


#               extraction des point (x,y) max et min
                maxx=a1.max()
                maxy=b2.max()
                minx=a1.min()
                miny=b2.min()
                # pose, scale , margin=pose(df)

                # ce teste et utilisé si les points qui calculent (x, y) ne sont pas visible alors on utilise le point
                # le plus haut extrait du Bbox de la persone pour fixer (y_head, x_head)
                if ((xs[0]==0)[0]) and ((xs[14]==0)[0]) and ((xs[15]==0)[0]) and ((xs[16]==0)[0]) and ((xs[17]==0)[0]):
                    df = pd.read_csv (a+do+'-bbox-'+str(n)+'.csv',header=None)
                    d=int(df[1])
                else :
                # pose, scale , margin=pose(df)
                    e,d=head(xs,ys)
                # d -> le point y de la tete
                # le point ys[13] -> la cheville d 
############### calcule de la pose et de la margin 
                pose = abs(ys[13]-d)
                scale =pose/240
                c=75
                margin=c*scale
                # t est utilisé pour décaler les points de l'extrémité 
                t= margin*0.4
                
            # w C'est le point bas-droite 
            # h C'est le point haut-gauche 

                h=abs(int(maxx-minx))

                w=abs(int(maxy-miny))

                import numpy as np
                import cv2
                h=int(h+margin)
                w=int(w+margin)
                image = cv2.imread(pathImg+do1+'_'+str(n)+'.jpg')
            # x C'est le point bas-gauche -> min de x ∈ au point relatif a la main droite
            # y C'est le point haut-droite -> min de y ∈ au point relatif a la main droite
                y=int(miny-t)
                x=int(minx-t)

                crop = image[y:y+h, x:x+w]
                crop = cv2.resize(crop, (224,224), interpolation = cv2.INTER_AREA)
                cv2.imwrite(r'C:\Users\amin\extracted patch\image\rh//'+mp+'//'+do+'_'+str(n)+'.jpg', crop)
                
                cv2.waitKey(0)



 10%|█         | 1/10 [09:12<1:22:51, 552.33s/it]


KeyboardInterrupt: 

## left hand

In [55]:


            
                

               


import pandas as pd
from PIL import Image
path =r'C:\Users\amin\extracted patch\Nouveau dossier'
dossier = [f for f in listdir(path) ]
m=0 
pathImg = r'C:\Users\amin\extracted patch\action'+'\\'
for i in tqdm(range (0,10)) :
    pathData = r'C:\Users\amin\extracted patch\Nouveau dossier'+'\\'
    pathData=pathData+ dossier[i]+'\\'
    Data = [f for f in listdir(pathData) ]
    mp=dossier[i]
    for j in range (0,len(Data)) :
        a=pathData
        do=Data[j]
        a=a+Data[j]+'\\'
        do1=do
        #ce test est utilisé parce que cette image n'est pas annotée correctement.
        if do1=='S10_punching_toLeft_sideView_HD':
            do1='S10_punching_toLeft__ideView_HD'
        nb=int((countNb(a)/2)+1)
                    ############### extraction du patche
        for n in  range(1,nb):
    
                 #lecture du fichier 
                df = pd.read_csv (a+do+'-pos_img-'+str(n)+'.csv',header=None)
                #ys c'est les points relatifs à laxe x
                #ys c'est les points relatifs à laxe y
                xs = df[df.index==0]
                ys = df[df.index==1]
                #concaténation des pt relatifs à l'extraction due patche Left hand
                a1=pd.concat([xs[6],xs[5],xs[7]])
                b2=pd.concat([ys[6],ys[5],ys[7]])
                
                
                ########  teste si un point est egale a 0 car il n est pas visible
                
                if (xs[5]==0)[0]:

                    a1=pd.concat([xs[6],xs[7]])
                    b2=pd.concat([ys[6],ys[7]])

                if  (xs[6]==0)[0]:

                    a1=pd.concat([xs[5],xs[7]])
                    b2=pd.concat([ys[5],ys[7]])

                if (xs[7]==0)[0]:

                    a1=pd.concat([xs[6],xs[5]])
                    b2=pd.concat([ys[6],ys[5]])
                    
#                     ____________________________

                if ((xs[5]==0)[0] and  (xs[6]==0)[0]):

                    a1=pd.concat([xs[7]])
                    b2=pd.concat([ys[7]])

                if ((xs[7]==0)[0] and (xs[6]==0)[0]):

                    a1=pd.concat([xs[5]])
                    b2=pd.concat([ys[5]])
                    

                if ((xs[7]==0)[0] and (xs[5]==0)[0]):

                    a1=pd.concat([xs[6]])
                    b2=pd.concat([ys[6]])
                    
                if ((xs[5]==0)[0]) and ((xs[6]==0)[0]) and  ((xs[7]==0)[0]):
                    df = pd.read_csv (a+do+'-pos_img-'+str(n-4)+'.csv',header=None)

                    xs = df[df.index==0]
                    ys = df[df.index==1]
                    a1=pd.concat([xs[5],xs[6],xs[7]])
                    b2=pd.concat([ys[5],ys[6],ys[7]])


                    if (xs[5]==0)[0]:

                        a1=pd.concat([xs[6],xs[7]])
                        b2=pd.concat([ys[6],ys[7]])

                    if  (xs[6]==0)[0]:

                        a1=pd.concat([xs[5],xs[7]])
                        b2=pd.concat([ys[5],ys[7]])

                    if (xs[7]==0)[0]:

                        a1=pd.concat([xs[5],xs[6]])
                        b2=pd.concat([ys[6],ys[5]])

    #         __________________________________________________________

                    if ((xs[5]==0)[0] )and ( (xs[6]==0)[0]):

                        a1=pd.concat([xs[7]])
                        b2=pd.concat([ys[7]])

                    if ((xs[5]==0)[0] )and ((xs[7]==0)[0]):

                        a1=pd.concat([xs[6]])
                        b2=pd.concat([ys[6]])


                    if ((xs[6]==0)[0]) and ((xs[7]==0)[0]):

                        a1=pd.concat([xs[5]])
                        b2=pd.concat([ys[5]])
   
               #               extraction des point (x,y) max et min
                    
                maxx=a1.max()
                maxy=b2.max()
                minx=a1.min()
                miny=b2.min()
                # pose, scale , margin=pose(df)

                # ce teste et utilisé si les points qui calculent (x, y) ne sont pas visible alors on utilise le point
                # le plus haut du bbox de la persone pour fixer (y_head, x_head)
                if ((xs[0]==0)[0]) and ((xs[14]==0)[0]) and ((xs[15]==0)[0]) and ((xs[16]==0)[0]) and ((xs[17]==0)[0]):
                    df = pd.read_csv (a+do+'-bbox-'+str(n)+'.csv',header=None)
                    d=int(df[1])
                else :
                # pose, scale , margin=pose(df)
                    e,d=head(xs,ys)
               # d -> le point y de la tete
                # le point ys[13] -> la cheville 
                pose = abs(ys[13]-d)
                 ############### calcule de la pose et de la margin 
                scale =pose/240
                c=75
                margin=c*scale



            
            
                 # t est utilisé pour décaler les points de l'extrémité 
                t= margin*0.4

            # w C'est le point bas-droite 
            # h C'est le point haut-gauche 
                h=abs(int(maxx-minx))

                w=abs(int(maxy-miny))

                import numpy as np
                import cv2
                h=int(h+margin)
                w=int(w+margin)
                image = cv2.imread(pathImg+do1+'_'+str(n)+'.jpg')
            # x C'est le point bas-gauche -> min de x ∈ au point relatif a la main gauche
            # y C'est le point haut-droite -> min de y ∈ au point relatif a la main gauche 

                y=int(miny-t)
                x=int(minx-t)

                crop = image[y:y+h, x:x+w]
                crop = cv2.resize(crop, (224,224), interpolation = cv2.INTER_AREA)
                cv2.imwrite(r'C:\Users\amin\extracted patch\image\lh//'+mp+'//'+do+'_'+str(n)+'.jpg', crop)
                
                cv2.waitKey(0)
                
                





  0%|          | 0/10 [01:58<?, ?it/s]


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\amin\\extracted patch\\Nouveau dossier\\S1\\S1_jogging_backView_HD\\S1_jogging_backView_HD-pos_img--2.csv'